In [ ]:
%loadFromPOM pom.xml

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import org.hl7.fhir.r4.model.*;
import java.util.Date;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

// 產生今天Schedule and Slot資料


In [ ]:
// search for a practitioner "陳莎莎"
String familyName = "陳";
String givenName = "莎莎";
int year = new Date().getYear(); 
int month = new Date().getMonth(); 
int day = new Date().getDate();

Bundle results = client
    .search()
    .forResource(Practitioner.class)
    .where(Practitioner.FAMILY.matches().value(familyName))
    .and(Practitioner.GIVEN.matches().value(givenName))
    .returnBundle(Bundle.class)
    .execute(); 
Practitioner practitioner = (Practitioner) results.getEntry().get(0).getResource();

// new schedule data for today
Schedule schedule = new Schedule();
schedule.addServiceType()
    .addCoding()
    .setSystem("http://terminology.hl7.org/CodeSystem/service-type")
    .setCode("124")
    .setDisplay("General Practice");
// service category
schedule.addServiceCategory()
    .addCoding()
    .setSystem("http://terminology.hl7.org/CodeSystem/service-category")
    .setCode("17")
    .setDisplay("General Practice");
// specialty
schedule.addSpecialty()
    .addCoding()
    .setSystem("http://snomed.info/sct")
    .setCode("394814009")
    .setDisplay("General practice service");
// actor    
schedule.addActor(new Reference("Practitioner/"+ practitioner.getIdElement().getIdPart()));
// planningHorizon

Period period = new Period();
period.setStart(new Date(year,month,day,8,00,00));
period.setEnd(new Date(year,month,day,17,00,00));
schedule.setPlanningHorizon(period);
// comment
schedule.setComment("陳莎莎醫師今日診所行程");
// print schedule

System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(schedule));

//save schedule
MethodOutcome outcome = client.create()
    .resource(schedule)
    .prettyPrint()
    .encodedJson()
    .execute();
//System.out.println(outcome.getId());
//System.out.println(outcome.getOperationOutcome().getIssue().get(0).getDiagnostics());


In [ ]:
// new all slots data for today, every sloy is 60 minutes
for(int i=8;i<17;i++){
    Slot slot = new Slot();
    slot.setSchedule(new Reference("Schedule/"+outcome.getId().getIdPart()));
    slot.addServiceType()
        .addCoding()
        .setSystem("http://terminology.hl7.org/CodeSystem/service-type")
        .setCode("124")
        .setDisplay("General Practice");
    slot.addServiceCategory()
        .addCoding()
        .setSystem("http://terminology.hl7.org/CodeSystem/service-category")
        .setCode("17")
        .setDisplay("General Practice");
    slot.addSpecialty()
        .addCoding()
        .setSystem("http://snomed.info/sct")
        .setCode("394814009")
        .setDisplay("General practice service");
    slot.setStart(new Date(year,month,day,i,00,00));
    slot.setEnd(new Date(year,month,day,i+1,00,00));
    slot.setStatus(Slot.SlotStatus.FREE);
    slot.setComment("陳莎莎醫師今日診所行程");
    // print slot
    System.out.println(ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(slot));
    // save slot
    MethodOutcome outcome2 = client.create()
        .resource(slot)
        .prettyPrint()
        .encodedJson()
        .execute();
}
